Prerequisites 

* PyFITS for reading fits files
* scikit-image (reading fits depends on PyFITS)


Or, you can use astropy.io.fits 


Note that PyFITS will give you a warning 'PyFITS is deprecated, please use astropy.io.fits.' But Scikit-image still depends on PyFITS :(

In [3]:
#import cv2
import numpy as np
import astropy
from astropy.io import fits
from astrobf.Shin import img_scale

import skimage
from skimage import io

In [4]:
# Parameters
ddir = "/home/hoseung/Work/tonemap/bf_data/examples/"
outdir = ddir
red_fn = ddir+"J000311.00+155754.0-i.fits"#   frame-i-007907-6-0143.fits"
green_fn = ddir+"J000311.00+155754.0-r.fits"#"frame-r-007907-6-0143.fits"
blue_fn = ddir+"J000311.00+155754.0-g.fits"#"frame-g-007907-6-0143.fits"
sig_fract = 5.0
per_fract = 5.0-2
max_iter = 20
sub_size_fract = 0.3
min_val = 0.0
red_factor = 1.0
green_factor = 1.0
blue_factor = 1.0
red_non_linear_fact = green_non_linear_fact = blue_non_linear_fact = 1
#green_non_linear_fact = 0.005
#blue_non_linear_fact = 0.005


In [5]:
def get_a_channel(fn, factor, return_size=True, **kwargs):
    from astropy.io import fits
    hdulist = fits.open(fn)
    img_header = hdulist[0].header
    img_data = np.array(hdulist[0].data, dtype=float)
    width, height = hdulist[0].data.shape
    hdulist.close()
    print("This file = {}, ({}, {})".format(fn, width, height))
    
    ##### find sub-samples for zscale
    flat_img_data = np.ravel(img_data)
    npix_tot = width * height
    sub_img_ind = np.random.randint(0, npix_tot, size=int(sub_size_fract*npix_tot))
    sub_img_ind = np.unique(sub_img_ind)
    sub_img_data = flat_img_data[sub_img_ind]
    print("size of the sample image = ", len(sub_img_data))
    min_v, max_v = img_scale.range_from_percentile(sub_img_data, low_cut=0.5, high_cut=0.25)
    red_factor * img_scale.asinh(img_data, scale_min = min_v, scale_max = max_v,
                                 **kwargs)
    if return_size:
        return factor * img_scale.asinh(img_data, **kwargs), (width, height)
    else:
        return factor * img_scale.asinh(img_data, **kwargs)
    
b, (width, height) = get_a_channel(blue_fn, blue_factor, non_linear=blue_non_linear_fact)
g, (width, height) = get_a_channel(green_fn, green_factor, non_linear=green_non_linear_fact)    
r, (width, height) = get_a_channel(red_fn, red_factor, non_linear=red_non_linear_fact)

This file = /home/hoseung/Work/tonemap/bf_data/examples/J000311.00+155754.0-g.fits, (181, 181)
size of the sample image =  8498
img_scale : asinh
img_scale : asinh
This file = /home/hoseung/Work/tonemap/bf_data/examples/J000311.00+155754.0-r.fits, (181, 181)
size of the sample image =  8509
img_scale : asinh
img_scale : asinh
This file = /home/hoseung/Work/tonemap/bf_data/examples/J000311.00+155754.0-i.fits, (181, 181)
size of the sample image =  8456
img_scale : asinh
img_scale : asinh


In [6]:
# RGB image with Scikit-image
print("image size ", width, height)
rgba_array = np.empty((width,height,4), np.uint8) # assuming 8 bits per channnel
rgba_array[:,:,0] = skimage.util.img_as_ubyte(r) # red
rgba_array[:,:,1] = skimage.util.img_as_ubyte(g) # green
rgba_array[:,:,2] = skimage.util.img_as_ubyte(b) # blue
rgba_array[:,:,3] = 255 # Alpha transparency
io.imsave(outdir+f'rgb_example_r{red_non_linear_fact:.2f}g{green_non_linear_fact:.2f}b{blue_non_linear_fact:.2f}.png',
          np.flipud(rgba_array))

image size  181 181
